In [2]:
# Visualize cameras:
from pytorch3d.vis.plotly_vis import _add_camera_trace
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from utils.camera_utils import read_camera_parameters
from pytorch3d.renderer import (
    FoVPerspectiveCameras,
    PerspectiveCameras
)

data_dir = "data/cow_data_10"
cam_scale = 0.1 # 10 for dance data, 0.1 for cow data
calibration_filepath = os.path.join(data_dir, "calibration.json")
Ks, Rs, ts = read_camera_parameters(calibration_filepath)
cameras_fov = FoVPerspectiveCameras(K=Ks, R=Rs, T=ts)
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])
_add_camera_trace(
                    fig, cameras_fov, "fov", 0, 1, cam_scale
                )
cameras_pers = PerspectiveCameras(K=Ks, R=Rs, T=ts)
_add_camera_trace(
                    fig, cameras_pers, "perspective", 0, 1, cam_scale
                )
fig.show()

In [40]:
import torch
from pytorch3d.transforms import so3_exp_map
# visualize novel cameras
logRs = torch.zeros(1, 3)
logRs[:, 1] = torch.linspace(-3.14, 3.14, 1)
Rs = so3_exp_map(logRs)
Ts = torch.zeros(1, 3)
Ts[:, 2] = 2.7

cameras = PerspectiveCameras(R=Rs, T=ts)
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])
_add_camera_trace(
                    fig, cameras, "name", 0, 1, 1
                )
fig.show()


In [4]:
from pytorch3d.renderer import look_at_view_transform
import numpy as np

Rs = []
Ts = []
for i in range(0,10):
    new_R, new_T = look_at_view_transform(dist=2.7, elev=0, azim=np.random.randint(0, 360))
    Rs.append(torch.squeeze(new_R))
    Ts.append(new_T)


Rs = torch.stack(Rs)
Ts = torch.squeeze(torch.stack(Ts, dim=0))

cameras = FoVPerspectiveCameras(R=Rs, T=Ts)
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])
_add_camera_trace(
                    fig, cameras, "name", 0, 1, 0.1
                )
fig.show()

In [48]:
from utils.camera_utils import (
    reshape_camera_matrices,
    read_camera_parameters,
)
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pytorch3d.renderer import (
    NDCMultinomialRaysampler,
    FoVPerspectiveCameras,
    MonteCarloRaysampler,
    FoVOrthographicCameras
)
from pytorch3d.vis.plotly_vis import (
    _add_ray_bundle_trace,
    _add_camera_trace
)
import os


data_dir = "data/cow_data"
calibration_filepath = os.path.join(data_dir, "calibration.json")
Ks, Rs, Ts = read_camera_parameters(calibration_filepath)
Ks, Rs, Ts = reshape_camera_matrices(Ks,Rs,Ts)
cam_id = 0
camera = FoVOrthographicCameras(R=Rs[cam_id].unsqueeze(0), T=Ts[cam_id].unsqueeze(0))

raysampler_grid = NDCMultinomialRaysampler(
        image_width=128,
        image_height=128,  
        n_pts_per_ray= 10, 
        min_depth= 10, # Note: should this be the same values as for training (min:0.1, max:3) ?
        max_depth= 3,
    )

raysampler_mc = MonteCarloRaysampler(
            min_x = -1.0,
            max_x = 1.0,
            min_y = -1.0,
            max_y = 1.0,
            n_rays_per_image=10,
            n_pts_per_ray=5,
            min_depth=0.1,
            max_depth=3,
        )
raybundle_grid = raysampler_grid(camera)
raybundle_mc = raysampler_mc(camera)

fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scene'}]])
_add_camera_trace(
                    fig, camera, "cam", 0, 1, 0.3
                )
_add_ray_bundle_trace(
                    fig=fig,
                    ray_bundle=raybundle_mc,
                    trace_name="raybundle",
                    subplot_idx=0,
                    ncols=1,
                    max_rays=10,
                    max_points_per_ray=128,
                    marker_size=3,
                    line_width=3,
                )

fig.show()

In [37]:

from pytorch3d.renderer import ray_bundle_to_ray_points
import plotly.express as px

fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scene'}]])
ray_points = ray_bundle_to_ray_points(raybundle_mc)


ray_points = (
    ray_bundle_to_ray_points(raybundle_mc)
)
print(ray_points.shape)
print(ray_points[:, :, :, 0].shape)
_add_camera_trace(
                    fig, camera, "cam", 0, 1, 0.3
                )
fig = go.Figure(data=go.Scatter3d(
        x=ray_points[:, :, :, 0],
        y=ray_points[:, :, :, 1],
        z=ray_points[:, :, :, 2],
        mode="markers",
        name="_points",
        marker={"size": 0.3},
    ))

fig.show()



torch.Size([1, 10, 5, 3])
torch.Size([1, 10, 5])
